In [ ]:
pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [ ]:
!pip install -q scann

In [ ]:
from typing import Dict, Text

import os
import pprint
import tempfile

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
import tensorflow_recommenders as tfrs

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as sp
from scipy.sparse import vstack
from scipy import sparse
from scipy.sparse.linalg import spsolve

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from pathlib import Path
import pandas as pd
import numpy as np
import random
from IPython.display import Image
from IPython.core.display import HTML 
import pdb;

from subprocess import check_output
from sklearn.model_selection import train_test_split
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

# Importing Libraries and cookbooks
from recsys import *## recommender system cookbook
#from generic_preprocessing import * ## pre-processing code
from IPython.display import HTML ## Setting display options for Ipython Notebook
from itertools import combinations, chain, product

In [ ]:
demo = pd.read_csv("demo001.csv",sep="|")

demo.head(10)

In [ ]:
from datetime import datetime
from datetime import date

In [ ]:
demo["Fecha_Nac"]=pd.to_datetime(demo["FECHA_NAC"], format='%Y-%m-%d %H:%M')
demo["Fecha_Nac"].head(5)

In [ ]:
hoy=datetime.now().year
def age(fecnan):
    return (hoy-fecnan.year)

demo["Edad"]=demo["Fecha_Nac"].apply(age)
demo["Edad"].head(5)

In [ ]:
demo["edad"]=pd.cut(demo['Edad'], [0,25,35,45,55,65,90]).astype(str)
demo.head()

In [ ]:
demo["NDepen"]=demo['N_D_EC'].apply(lambda x: 0 if x == 0 else 1).astype(str)
demo.head()

In [ ]:
df_demo=demo.loc[:,('ID_CTE', "GENERO", 'EDO_CIVIL',"edad","NDepen","TP_HOGAR")]
df_demo["user_id"]=df_demo["ID_CTE"].astype(str)
df_demo.head(10)

In [ ]:
tran = pd.read_csv("transactional001.csv",sep="|")

tran.head(10)

In [ ]:
df_tran=tran[((tran["PROD_AREA"]=="Muebles")  | (tran["PROD_AREA"]=="Ropa"))&((tran["IMPORTE_VTA"]>100))]


In [ ]:
df_tran["area"]=df_tran["PROD_AREA"].apply(lambda x: 1 if x == 'Ropa' else 2) 
df_tran["fam"]=df_tran["area"].astype(str)+df_tran["PROD_DEP"].astype(str)+df_tran["PROD_CLAS"].astype(str).str.zfill(2)+df_tran["PROD_FAM"].astype(str).str.zfill(3)
df_tran["clas"]=df_tran["area"].astype(str)+df_tran["PROD_DEP"].astype(str)+df_tran["PROD_CLAS"].astype(str).str.zfill(2)
df_tran["idclas"]=df_tran["clas"]
df_tran["idfam"]=df_tran["fam"]
df_tran["movie_id"]=df_tran["fam"].astype(str)#df_tran["DESC_PROD_DEP"]#
df_tran["user_id"]=df_tran["ID_CTE"].astype(str)

In [ ]:
df_f=df_tran.loc[:,('user_id', 'FECHA_TICKET', 'movie_id')].groupby(['user_id',"movie_id"]).size().reset_index()
df_f.head(10)

In [ ]:
df_f1=df_f[:50000]

In [ ]:
dff1=pd.merge(df_f1, df_demo, on='user_id', how='inner')

dff1.head(10)

In [ ]:
df_1=dff1.dropna() 
df_1.head(10)

In [ ]:
dff=pd.DataFrame(df_1["movie_id"].unique())
dff["movie_id"]=dff[0].astype(str)
dff.head()

In [ ]:
prod = pd.read_csv("prod_variables.csv",sep=",")

prod.head(10)

In [ ]:
prod["movie_id"]=prod["id_Familia"].astype(str)
prod["id_Familia"]=prod["id_Familia"].astype(str)
prod.head()

In [ ]:
dfff=df_1.loc[:,("user_id","movie_id","GENERO","EDO_CIVIL","edad","NDepen","TP_HOGAR")]
dfff.head(5)

In [ ]:
dfp=pd.merge(dfff, prod, on='movie_id', how='inner')

dfp.head(10)

In [ ]:
dfpp=dff["movie_id"].unique()#pd.DataFrame({"movie_id" : dff.loc[:,("movie_id")]})#dff["movie_id"].unique()#dfp.loc[:,"movie_id"]
dfpp

In [ ]:
dfff=df_1.loc[:,("user_id","movie_id","GENERO","EDO_CIVIL","edad","NDepen","TP_HOGAR")]
dfff.head(5)

In [ ]:
dfp=pd.merge(prod,dfff, on='movie_id', how='inner')#df_1.loc[:,("user_id","movie_id","GENERO","EDO_CIVIL","edad","NDepen","TP_HOGAR")]
dfp.head(5)

In [ ]:
dfp=pd.merge(prod,dfff, on='movie_id', how='inner')#df_1.loc[:,("user_id","movie_id","GENERO","EDO_CIVIL","edad","NDepen","TP_HOGAR")]
dfp.head(5)

In [ ]:
dfmo=pd.DataFrame({"movie_id" : dff.loc[:,("movie_id")]})#dff["movie_id"].unique()#dfp.loc[:,"movie_id"]
dfmovie=pd.merge(prod,dfmo, on='movie_id', how='inner')
dfmovie.head()

In [ ]:
ratings = tf.data.Dataset.from_tensor_slices((dfp.to_dict('list')))

In [ ]:
ratings.take(10)

In [ ]:
dfr = tfds.as_dataframe(ratings.take(10))
dfr.head()

In [ ]:
movies= tf.data.Dataset.from_tensor_slices((dfmovie.to_dict('list')))

In [ ]:
dfm = tfds.as_dataframe(movies.take(10))
dfm.head()

In [ ]:
user_ids_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))
#y=ratings.map(lambda x: x["user_id"])

movie_titles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies.map(lambda x: x["movie_id"]))

In [ ]:
#timestamps = np.concatenate(list(ratings.map(lambda x: x["GENERO"].astype(int)).batch(50)))

#max_timestamp = timestamps.max()
#min_timestamp = timestamps.min()

#timestamp_buckets = np.linspace(
 #   min_timestamp, max_timestamp, num=100,
#)

timestamps= np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["GENERO"]))))

EC= np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["EDO_CIVIL"]))))

ED= np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["edad"]))))

DE= np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["NDepen"]))))


TP= np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["TP_HOGAR"]))))

cat= np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["Categoria"]))))

fam= np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["Familia"]))))


unique_movie_titles = np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["movie_id"]))))
unique_user_ids = np.unique(np.concatenate(list(ratings.batch(5_0).map(
    lambda x: x["user_id"]))))

In [ ]:
class UserModel(tf.keras.Model):
  
  def __init__(self, use_timestamps):
    super().__init__()

    self._use_timestamps = use_timestamps

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])

    if use_timestamps:
      self.timestamp_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=timestamps, mask_token=None),
      tf.keras.layers.Embedding(len(timestamps) + 1, 32)
        ])
    
      self.EC_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=EC, mask_token=None),
      tf.keras.layers.Embedding(len(EC) + 1, 32)
        ])
        
      self.ED_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=ED, mask_token=None),
      tf.keras.layers.Embedding(len(ED) + 1, 32)
        ])
    
      self.DE_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=DE, mask_token=None),
      tf.keras.layers.Embedding(len(DE) + 1, 32)
        ])
        
      self.TP_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=TP, mask_token=None),
      tf.keras.layers.Embedding(len(TP) + 1, 32)
       ])

  def call(self, inputs):
    if not self._use_timestamps:
      return self.user_embedding(inputs["user_id"])
    #tf.print(inputs["user_id"])
    return tf.concat([
            self.user_embedding(inputs["user_id"]),
            self.timestamp_embedding(inputs["GENERO"]),
            self.EC_embedding(inputs["EDO_CIVIL"]),
            self.ED_embedding(inputs["edad"]),
            self.DE_embedding(inputs["NDepen"]),
            self.TP_embedding(inputs["TP_HOGAR"])
            ], axis=1)

In [ ]:
class MovieModel(tf.keras.Model):
  
  def __init__(self,use_timestamps):
    super().__init__()
    self._use_timestamps = use_timestamps
    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
          vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, 32)
    ])
    
    self.title_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
        max_tokens=max_tokens)

    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    #self.title_vectorizer.adapt(movies)
    self.title_vectorizer.adapt(ratings.map(lambda x: x["movie_id"]))
    if use_timestamps:
        self.cat_embedding = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=cat, mask_token=None),
        tf.keras.layers.Embedding(len(cat) + 1, 32)
        ])
        
        self.fam_embedding = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=fam, mask_token=None),
        tf.keras.layers.Embedding(len(fam) + 1, 32)
        ])
    
   # def call(self, inputs):
    #if not self._use_timestamps:
     # return self.title_embedding(titles["movie_id"]),
      #  self.title_text_embedding(titles["movie_id"])
    
  def call(self, titles):
    if not self._use_timestamps:
      return tf.concat([
            self.title_embedding(titles["movie_id"]),
            self.title_text_embedding(titles["movie_id"]),
        ], axis=1)

      #tf.print(titles["movie_id"])
    return tf.concat([
            self.title_embedding(titles["movie_id"]),
            self.title_text_embedding(titles["movie_id"]),
            self.cat_embedding(titles["Categoria"]),
            self.fam_embedding(titles["Familia"])
            ], axis=1)

In [ ]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, use_timestamps):
    super().__init__()
    self.query_model = tf.keras.Sequential([
      UserModel(use_timestamps),
      tf.keras.layers.Dense(32)
    ])
    self.candidate_model = tf.keras.Sequential([
      MovieModel(use_timestamps),
      tf.keras.layers.Dense(32)
    ])
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"],
        "GENERO": features["GENERO"],
        "EDO_CIVIL": features["EDO_CIVIL"],
        "edad": features["edad"],
         "NDepen": features["NDepen"],
         "TP_HOGAR": features["TP_HOGAR"]
                                            })
    #movie_embeddings = self.candidate_model(features["movie_id"])
    
    movie_embeddings = self.candidate_model({
       "movie_id": features["movie_id"],
       "Categoria": features["Categoria"],
        "Familia": features["Familia"]
                                                 
                                                 })
    
    #movie_embeddings = self.candidate_model(features["movie_title"])

    return self.task(query_embeddings, movie_embeddings)

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(1_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(8_000)
test = shuffled.skip(8_000).take(2_000)

cached_train = train.shuffle(1_000).batch(204)
cached_test = test.batch(406).cache()

In [ ]:
model = MovielensModel(use_timestamps=False)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

In [ ]:
model = MovielensModel(use_timestamps=True)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"accuracy (train): {train_accuracy:.2f}.")
print(f"accuracy (test): {test_accuracy:.2f}.")